# Recomendação de músicas e filmes

### Movies Dataset

#### Importação de bibliotecas

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode, from_json
from pyspark.sql.types import StringType, ArrayType, IntegerType, StructType, StructField
from re import sub

#### Constantes

In [0]:
# Caminho dos datasets
#movies_metadata_path = "../datasets/movies_metadata.csv"
#movies_credits_path = "../datasets/movies_credits.csv"
movies_metadata_path = "dbfs:/FileStore/shared_uploads/beatrizpatricio@estudante.ufscar.br/movies_metadata.csv"
movies_credits_path = "dbfs:/FileStore/shared_uploads/beatrizpatricio@estudante.ufscar.br/credits.csv"

# Conexão com o Neo4j
url = "neo4j://localhost:7687"
username = "neo4j"
password = ""
dbname = ""
connector_path = ""

# Meu Neo4j
neo4j_url = "neo4j+s://90016f46.databases.neo4j.io"  # ou bolt+s://<host>.databases.neo4j.io para AuraDB neo4j+s://90016f46.databases.neo4j.io
neo4j_user = "neo4j"
neo4j_password = "iA5r3A7HdNjl084_m47abMkDF6SlUpm1n2jVG7bR6HY"


#### Preparação da conexão com o Neo4j

In [0]:
spark = (
    SparkSession.builder.config("neo4j.url", neo4j_url)
    .config("neo4j.authentication.basic.username", neo4j_user)
    .config("neo4j.authentication.basic.password", neo4j_password)
    .config("neo4j.database", dbname)
    .getOrCreate()
)

### 1. Leitura do dataset de metadata

In [0]:
def getMetadataDataset ():
    df_metadata = (spark.read
        .format("csv")
        .option("header", True)
        .option("inferSchema", True)
        .load(movies_metadata_path)
    )

    return df_metadata

df_metadata = getMetadataDataset()
df_metadata.printSchema()
print(f"{df_metadata.count()} linhas")
df_metadata.first()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

### 2. Criação da coluna de identificador

In [0]:
def getIdentifierName (title):
    if title:
        return sub(r'[^a-zA-Z0-9]', '', title).lower().strip()
    return None

def addIdentifierColumn (df_metadata):
    getIdentifierNameUdf = udf(getIdentifierName, StringType())
    return df_metadata.withColumn("identifierByName", getIdentifierNameUdf(df_metadata['title'])) \
        .withColumn("movie_id", col("id"))

df_metadata = addIdentifierColumn(df_metadata)
print(f"Exemplo de identificador para o filme '{df_metadata.first()['title']}': {df_metadata.first()['identifierByName']}")

Exemplo de identificador para o filme 'Toy Story': toystory


### 3. Remoção de colunas que não interessam para o projeto

_As colunas remanescentes são: id, title, genres, identifierByName e imdb\_id_

In [0]:
def dropColumns (df_metadata):
    columnsToDrop = [
        "adult",
        "belongs_to_collection",
        "budget",
        "homepage",
        "original_language",
        "original_title",
        "overview",
        "popularity",
        "poster_path",
        "production_companies",
        "production_countries",
        "revenue",
        "runtime",
        "spoken_languages",
        "status",
        "tagline",
        "video",
        "vote_average",
        "vote_count",
        "id",
        "imdb_id"
    ]

    return df_metadata.drop(*columnsToDrop)

df_metadata = dropColumns(df_metadata)
df_metadata.printSchema()
print()
print(df_metadata.first())

root
 |-- genres: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- identifierByName: string (nullable = true)
 |-- movie_id: string (nullable = true)


Row(genres="[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]", release_date='1995-10-30', title='Toy Story', identifierByName='toystory', movie_id='862')


### 4. Remoção de linhas sem título ou identificador, e com data de lançamento mal-formada

In [0]:
def dropRows (df_metadata):
    requiredColumns = [
        "title",
        "identifierByName",
    ]

    df_metadata = df_metadata.distinct()
    df_metadata = df_metadata.na.drop("all", subset=requiredColumns)
    return df_metadata.filter(~col("release_date").rlike(r'\D{4}-\D{2}-\D{2}'))

print(df_metadata.count(), " linhas originalmente")
df_metadata = dropRows(df_metadata)
print(df_metadata.count(), " linhas após remoção")

45572  linhas originalmente
44673  linhas após remoção


### 5. Leitura do dataset de créditos

In [0]:
def getCreditsDataset():
    df_credits = (
        spark.read
        .format("csv")
        .option("mode", "DROPMALFORMED")
        .option("header", True)
        .option("inferSchema", True)
        .load(movies_credits_path)
    )

    return df_credits.filter(~col("id").rlike(r'\D+'))

df_credits = getCreditsDataset()
df_credits.printSchema()
print(f"\t{df_credits.count()} linhas")

root
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)
 |-- id: string (nullable = true)

	30457 linhas


### 6. Criação do dataframe de Diretores/Filmes

In [0]:
def getDirectorsDataFrame (df_credits):
    crew_schema = ArrayType(
        StructType([
            StructField("credit_id", StringType()),
            StructField("department", StringType()),
            StructField("gender", IntegerType()),
            StructField("id", IntegerType()),
            StructField("job", StringType()),
            StructField("name", StringType()),
            StructField("profile_path", StringType())
        ])
    )

    df_with_crew = df_credits.withColumn("crew_array", from_json(col("crew"), crew_schema))
    df_exploded = df_with_crew.withColumn("crew_member", explode("crew_array"))

    return (
        df_exploded
        .filter(col("crew_member.job") == "Director")
        .select(
            col("crew_member.name").alias("director_name"),
            col("id").alias("movie_id")
        )
    )

df_directors = getDirectorsDataFrame(df_credits)
df_directors.show(truncate=False)
print(f"\t{df_directors.count()} linhas")

+------------------+--------+
|director_name     |movie_id|
+------------------+--------+
|Oliver Parker     |16420   |
|Richard Loncraine |31174   |
|Claude Lelouch    |48750   |
|Jafar Panahi      |46785   |
|Henry Jaglom      |188588  |
|Nick Castle       |47475   |
|Diane Keaton      |52856   |
|Phillip Borsos    |27985   |
|Art Clokey        |43475   |
|J. F. Lawton      |32631   |
|David Frankel     |17402   |
|Gregory Nava      |38722   |
|Mario Van Peebles |41478   |
|Peter Yates       |161495  |
|Wallace Wolodarsky|32502   |
|Robert Wuhl       |203119  |
|Michael Corrente  |171857  |
|Thomas Schlamme   |38129   |
|Whit Stillman     |16771   |
|Michael Apted     |26203   |
+------------------+--------+
only showing top 20 rows

	3649 linhas


### 7. Criação do dataframes de gêneros
_Também é dropada a coluna de gêneros do dataframe original_

In [0]:
def getGenres(df_metadata):
  genres_schema = ArrayType(
    StructType([
      StructField("id", IntegerType(), True),
      StructField("name", StringType(), True)
    ])
  )

  df_parsed = df_metadata.withColumn("genres_json", from_json(col("genres"), genres_schema))
  df_exploded = df_parsed.withColumn("genre", explode(col("genres_json")))

  df_genres_movies = df_exploded.select(
    col("movie_id"),
    col("genre.id").alias("genre_id"),
    col("genre.name").alias("genre_name")
  )

  df_genres = df_genres_movies.select("genre_id", "genre_name").distinct()

  return df_genres_movies, df_genres

df_genres_movies, df_genres = getGenres(df_metadata)
df_metadata = df_metadata.drop("genres")

df_genres_movies.show(truncate=False)
print(f"\t{df_genres_movies.count()} linhas")

df_genres.show(truncate=False)
print(f"\t{df_genres.count()} linhas")

+--------+--------+----------+
|movie_id|genre_id|genre_name|
+--------+--------+----------+
|10451   |35      |Comedy    |
|10451   |18      |Drama     |
|10451   |10749   |Romance   |
|11566   |35      |Comedy    |
|17600   |35      |Comedy    |
|17600   |18      |Drama     |
|38884   |18      |Drama     |
|38884   |36      |History   |
|38884   |10752   |War       |
|38884   |10749   |Romance   |
|46063   |18      |Drama     |
|46063   |53      |Thriller  |
|9802    |28      |Action    |
|9802    |12      |Adventure |
|9802    |53      |Thriller  |
|20318   |18      |Drama     |
|20318   |36      |History   |
|23114   |18      |Drama     |
|23114   |10751   |Family    |
|32144   |18      |Drama     |
+--------+--------+----------+
only showing top 20 rows

	89542 linhas
+--------+---------------+
|genre_id|genre_name     |
+--------+---------------+
|878     |Science Fiction|
|28      |Action         |
|35      |Comedy         |
|9648    |Mystery        |
|10769   |Foreign        |


### 8. Criação do Dataframe de filmes (união do df de metadata e de diretores)
_Também é dropada a coluna de movie_id do dataframe de diretores_

In [0]:
def getFinalMovieDataFrame (df_metadata, df_directors):
    return df_metadata.join(
        df_directors,
        on = df_metadata["movie_id"] == df_directors["movie_id"],
        how = "left"
    )

df_movies = getFinalMovieDataFrame(df_metadata, df_directors)
df_directors = df_directors.drop("movie_id")

In [0]:
df_movies.printSchema()
print(df_movies.first())
print()
print(f"Quantidade total de linhas: {df_movies.count()}")

root
 |-- release_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- identifierByName: string (nullable = true)
 |-- movie_id: string (nullable = true)
 |-- director_name: string (nullable = true)
 |-- movie_id: string (nullable = true)

Row(release_date='1995-10-30', title='Toy Story', identifierByName='toystory', movie_id='862', director_name=None, movie_id=None)

Quantidade total de linhas: 44826


### Spotify dataset 

### Trilha sonora dataset 

In [0]:
df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/beatrizpatricio@estudante.ufscar.br/sound_track_imdb_top_250_movie_tv_series.csv")

In [0]:
# Substitui "NA" por None em todas as colunas
df = df.replace("NA", None)

In [0]:
from pyspark.sql.functions import col, sum

null_counts = df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in df.columns
])

null_counts.show()

In [0]:
# Seleciona as primeiras 6 colunas (remove as colunas que não serão utilizadas e possuem muitos nulos)
selected_columns = df.columns[:6]
df = df.select(*selected_columns)

In [0]:
from pyspark.sql.functions import regexp_replace
# remove a marcação "(uncredited)" da coluna performed_by
df = df.withColumn("performed_by", regexp_replace("performed_by", r"\(uncredited\)", "").alias("performed_by"))

In [0]:
%sql
use catalog spark_catalog;
use schema default;
show tables

In [0]:
spark.sql("USE CATALOG spark_catalog")
spark.sql("USE SCHEMA default")  # ou DATABASE

In [0]:
df.write.saveAsTable("spark_catalog.default.sound_track_table")

### 8. Inserção no neo4j

In [0]:
df_genres.write.format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("labels", ":MovieGenre") \
    .option("node.keys", "genre_id") \
    .save()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1277578922690285>:1
----> 1 df_genres.write.format("org.neo4j.spark.DataSource") \
      2     .mode("Overwrite") \
      3     .option("labels", ":MovieGenre") \
      4     .option("node.keys", "genre_id") \
      5     .save()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1395, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1393     self.format(format)
   1394 if path is None:
-> 1395     self._jwrite.save()
   13

In [0]:
df_directors.write.format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("labels", ":Director") \
    .option("node.keys", "director_name") \
    .save()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1277578922690286>:1
----> 1 df_directors.write.format("org.neo4j.spark.DataSource") \
      2     .mode("Overwrite") \
      3     .option("labels", ":Director") \
      4     .option("node.keys", "director_name") \
      5     .save()

NameError: name 'df_directors' is not defined

In [0]:
df_metadata.write.format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("labels", ":Movie") \
    .option("node.keys", "movie_id") \
    .save()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1277578922690287>:1
----> 1 df_metadata.write.format("org.neo4j.spark.DataSource") \
      2     .mode("Overwrite") \
      3     .option("labels", ":Movie") \
      4     .option("node.keys", "movie_id") \
      5     .save()

NameError: name 'df_metadata' is not defined

In [0]:
df_genres_movies = df_genres_movies.coalesce(1)

df_genres_movies.write.format("org.neo4j.spark.DataSource") \
    .mode("Append") \
    .option("batch.size", "100") \
    .option("maxTransactionRetryTime", "30s") \
    .option("relationship", "HAS_GENRE") \
    .option("relationship.save.strategy", "keys") \
    .option("relationship.source.labels", ":Movie") \
    .option("relationship.target.labels", ":MovieGenre") \
    .option("relationship.source.node.keys", "movie_id") \
    .option("relationship.target.node.keys", "genre_id") \
    .save()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1277578922690288>:1
----> 1 df_genres_movies = df_genres_movies.coalesce(1)
      3 df_genres_movies.write.format("org.neo4j.spark.DataSource") \
      4     .mode("Append") \
      5     .option("batch.size", "100") \
   (...)
     12     .option("relationship.target.node.keys", "genre_id") \
     13     .save()

NameError: name 'df_genres_movies' is not defined

In [0]:
df_movies = df_movies.coalesce(1)

df_movies.write.format("org.neo4j.spark.DataSource") \
    .mode("Append") \
    .option("batch.size", "100") \
    .option("maxTransactionRetryTime", "30s") \
    .option("relationship", "DIRECTED") \
    .option("relationship.save.strategy", "keys") \
    .option("relationship.source.labels", ":Movie") \
    .option("relationship.target.labels", ":Director") \
    .option("relationship.source.node.keys", "movie_id") \
    .option("relationship.target.node.keys", "director_name") \
    .save()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1277578922690289>:1
----> 1 df_movies = df_movies.coalesce(1)
      3 df_movies.write.format("org.neo4j.spark.DataSource") \
      4     .mode("Append") \
      5     .option("batch.size", "100") \
   (...)
     12     .option("relationship.target.node.keys", "director_name") \
     13     .save()

NameError: name 'df_movies' is not defined